In [26]:
import cv2
import numpy as np
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import imutils

In [60]:
def create_data_set(x_crop= 150, y_crop=150, train_size=.8): 
    
    """ Load the Yale Faces data set, extract the faces on the images and generate labels for each image.
        
        Returns: Train and validation samples with their labels. The training samples are flattened arrays 
        of size 22500 (150 * 150) , the labels are one-hot-encoded values for each category
    """
    images_path = [ os.path.join("yalefaces", item)  for item in  os.listdir("yalefaces") ]
    image_data = []
    image_labels = []
    
    for i,im_path in enumerate(images_path):
        
        if os.path.isfile(im_path) : 
            
            #im = io.imread(im_path,as_gray=True)
            im = io.imread(im_path)
            im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
            #im = imutils.resize(im, width=600)

            

            #image_data.append(np.array(im, dtype='uint8'))
            image_data.append(im)


            label = int(os.path.split(im_path)[1].split(".")[0].replace("subject", ""))  -1


            image_labels.append(label)
        
    return image_labels, image_data
    

In [61]:
image_labels, image_data = create_data_set(x_crop= 150, y_crop=150, train_size=.8)

In [62]:
image_data[2].shape

(243, 320, 3)

In [63]:
name_MatImage = dict(zip(image_labels, image_data))

In [15]:
len(image_labels)

166

In [5]:
def embedder():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-2332150aa2c2>, line 2)

In [64]:
print("[INFO] loading face detector")
protoPath = os.path.sep.join(["../model", "deploy.prototxt"])
modelPath = os.path.sep.join(["../model","res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch("../embedder/openface_nn4.small2.v1.t7")

[INFO] loading face detector


In [66]:
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

# loop over the image paths
for name, Mat in name_MatImage.items():

    
	(h, w) = Mat.shape[:2]
	
	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(Mat, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=False, crop=False)
	#print(imageBlob.shape)
	#cv2.imwrite("./test.jpg", imageBlob)
	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		print(detections)
		break
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = Mat[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1



[[[[ 0.00000000e+00  1.00000000e+00  9.99804676e-01  3.87408078e-01
     2.14161068e-01  7.81234562e-01  9.05394197e-01]
   [ 0.00000000e+00  1.00000000e+00  5.23800910e-01  7.19001889e-01
     5.30574203e-01  1.10728586e+00  1.25078237e+00]
   [ 0.00000000e+00  1.00000000e+00  1.13814317e-01  1.60349876e-01
     4.01014996e+00  8.12743664e-01  4.98805332e+00]
   [ 0.00000000e+00  1.00000000e+00  1.07703060e-01  4.17965889e+00
     4.00302410e+00  4.82558632e+00  4.98468781e+00]
   [ 0.00000000e+00  1.00000000e+00  1.03982456e-01  1.87996626e-02
     1.17988825e-01  9.68350649e-01  8.71447027e-01]
   [ 0.00000000e+00  1.00000000e+00  1.03316851e-01  4.02680731e+00
     1.19631141e-01  4.97647047e+00  8.73659134e-01]
   [ 0.00000000e+00  1.00000000e+00  8.90385732e-02  5.26490211e-01
     5.48904538e-01  6.59391165e-01  7.15391874e-01]
   [ 0.00000000e+00  1.00000000e+00  8.46713111e-02  4.14013529e+00
     1.06232250e+00  4.82620096e+00  1.95530164e+00]
   [ 0.00000000e+00  1.00000000e

In [67]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(args["embeddings"], "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 0 encodings...


NameError: name 'args' is not defined

In [24]:
def send_log(log_file):
    """
    send mail using postfix 
    """
    return 'echo "log : \n\nAmplitude Daily Extraction \n\nLeads and MAU/DAU \n" | mutt -s "log : Amplitude Daily Extraction" bi@avito.ma -a '


In [25]:
a = send_log("")
a

'echo "log : \n\nAmplitude Daily Extraction \n\nLeads and MAU/DAU \n" | mutt -s "log : Amplitude Daily Extraction" bi@avito.ma -a '

In [23]:
print(a)

None
